# KNN Classifier

In [ ]:
!pip install sklearn

In [ ]:
import pandas as pd
import numpy as np

1) Get the dataset Adult (either in the UCI Repository or in Kaggle).

Usaremos o file_input para dar drop em linhas que possuam valores vazios e file_input_entropy para ajustar aos valores mais frequentes de cada feature

In [ ]:
file_input.drop(columns=['fnlwgt'], inplace=True, axis=1)
file_input_entropy.drop(columns=['fnlwgt'], inplace=True, axis=1)

2) Analyze features, missing data, and overall characteristics of the dataset (using tools in sklearn). Select features, discretize numerical features, handle missing data.


Para tratar os dados, usaremos 2 formas e comparaçoe.
1- Exclusao das linhas com valores nulos.
2- Tratamento da informaçao nula e substituiçao pelo valor mais provavel ( Considerando a moda da feature)

In [ ]:
#Drop de linhas com workclass vazia.
file_input.drop(file_input.index[(file_input['workclass']=='?')],axis=0,inplace=True)

In [ ]:
file_input.drop(file_input.index[(file_input['occupation']=='?')],axis=0,inplace=True)

In [ ]:
file_input.drop(file_input.index[(file_input['native.country']=='?')],axis=0,inplace=True)


Precisamos Normalizar a base que tem campos categóricos. Utilizaremos o OneHotEncoder para isso. Cada categoria vira uma coluna nova.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
x_full_entropy=file_input_entropy.drop(['income'],axis=1)
y_full_entropy=file_input_entropy['income']
y_ent=file_input_entropy['income']
y_label = {'<=50K': 0, '>50K': 1}
y_entropy= y_ent.replace(y_label)
y=file_input['income']
y_label = {'<=50K': 0, '>50K': 1}
y = y.replace(y_label)
y.describe

In [ ]:
index_categorico = [k == 'O' for k in x_full_entropy.dtypes]
x_full_entropy.iloc[:, index_categorico]
colunas_categoricas = x_full_entropy.columns[index_categorico]

In [ ]:
# tratamento de nulos do file_input_entropy
for k in colunas_categoricas:
    index2=file_input_entropy.index[(file_input_entropy[k]=='?')]
    for i2 in index2:
        file_input_entropy.replace(file_input_entropy[k][i2],file_input_entropy[k].max(), inplace=True)

In [ ]:
X_categorias_entropy = file_input_entropy.drop(['income'],axis=1).iloc[:, index_categorico]
ohe_entropy = OneHotEncoder(dtype=int).fit(X_categorias_entropy)
X_ohe_entropy = pd.DataFrame(ohe_entropy.transform(X_categorias_entropy).toarray(), columns=ohe_entropy.get_feature_names_out())
print(X_ohe_entropy.shape)
X_ohe_entropy.head()

In [ ]:
from sklearn.model_selection import train_test_split
p = 0.3 # fraction of elements in the test set
x_train_entropy, x_test_entropy, y_train_entropy, y_test_entropy = train_test_split(X_ohe_entropy,y_entropy, test_size = p)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
k = 6
model = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
model.fit(x_train_entropy, y_train_entropy)
y_pred_knn_com_tratamentoNull = model.predict(x_test_entropy)

In [ ]:
from sklearn.metrics import accuracy_score
score_knn_tratamento = accuracy_score(y_pred_knn_com_tratamentoNull, y_test_entropy)
print('Acuracia KNN',str(k), ' Com tratamento de dados faltantes (Coloca o de maior frequência da feature) ', ':', score)

Acuracia KNN 6  Com tratamento de dados faltantes (Coloca o de maior frequência da feature)  : 0.8197358992732112


In [ ]:
# cross validation com
from sklearn.model_selection import cross_val_score
from statistics import mean
scores = cross_val_score(model,X_ohe_entropy,y_entropy, cv=5)
print("media acuracia usando cross validation: " + str(mean(scores)))

media acuracia usando cross validation: 0.8119526256502304


### Tratamento de valores categoricos com o OneHotEncoder Para base com drop em linhas com informação faltante

In [ ]:
index_categorico = [k == 'O' for k in x_full.dtypes]
x_full.iloc[:, index_categorico]
colunas_categoricas = x_full.columns[index_categorico]

In [ ]:
from sklearn.preprocessing import OneHotEncoder
X_categorias = x_full.iloc[:, index_categorico]
ohe = OneHotEncoder(dtype=int).fit(X_categorias)
X_ohe = pd.DataFrame(ohe.transform(X_categorias).toarray(), columns=ohe.get_feature_names_out())
print(X_ohe.shape)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
p = 0.3 # fraction of elements in the test set
x_train, x_test, y_train, y_test = train_test_split(X_ohe,y, test_size = p)

In [ ]:
model = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
model.fit(x_train, y_train)
y_pred_knn_notNulls = model.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
score_knn_notNulls = accuracy_score(y_pred_knn_notNulls, y_test)
print('Acuracia KNN -',str(k), ' Desprezando as linhas com nulos:', score_knn_notNulls)

Acuracia KNN - 6  Desprezando as linhas com nulos: 0.7992043319703834


In [ ]:
from sklearn.model_selection import cross_val_score
from statistics import mean
clf = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
scores = cross_val_score(clf,X_ohe,y, cv=5)
print("media acuracia usando cross validation: " + str(mean(scores)))

# Other classifiers

Iremos utilizar alguns outros classificadores para comparação.

### Support Vector Machine

In [ ]:
from sklearn import svm
regr = svm.SVC()
regr.fit(x_train_entropy, y_train_entropy)
y_pred_svm=regr.predict(x_test_entropy)
score_svm = accuracy_score(y_pred_svm, y_test_entropy)

### Randon forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
random_forest_class= RandomForestClassifier(max_depth=20, random_state=0)
random_forest_class.fit(x_train_entropy, y_train_entropy)
y_pred_random_forest=random_forest_class.predict(x_test_entropy)
score_random_forest = accuracy_score(y_pred_random_forest, y_test_entropy)
print(score_random_forest)

### Regressão Logistica

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV(cv=5, random_state=0).fit(x_train_entropy, y_train_entropy)
y_pred_logistic=clf.predict(x_test_entropy)
score_logistic = accuracy_score(y_pred_logistic, y_test_entropy)
print(score_logistic)

### XBoost

In [ ]:
import xgboost as xgb
xgb_cl = xgb.XGBClassifier()
xgb_cl.fit(x_train_entropy, y_train_entropy)
y_pred_xboost = xgb_cl.predict(x_test_entropy)
score_xboost = accuracy_score(y_pre_xboost, y_test_entropy)
print(score_xboost)

# Resultados obtidos
Nesse tópico iremos comparar a precisão, acurácia e outras métricas dos modelos e técnicas urtilizadas para classificação.
- Comparação de acurácia de cada modelo.
- Comparação de F_{1} Score.
- Comparação de precisão, recall, F_score e suporte.
O código de cálculo dos indicadores e métricas para cada classificador, ficou grande e teve que ser suprimido.

In [ ]:
print("F1 Score para todos os classificadores" + str(f1_values))

F1 Score para todos os classificadores
{'Regressao logistica': 0.5941361763993216, 'Knn': 0.5410122164048866, 'SVM': 0.5952208544532946, 'Random forest': 0.19506233194817893, 'XGBoost': 0.615020137408197}


In [ ]:
print("Precisao, recall, Fscore e suporte para todos os classificadores" + str(fs_values))

Precisao, recall, Fscore e suporte para todos os classificadores
{'Regressao logistica': (0.5175179400590967, 0.6973833902161547, 0.5941361763993216, None), 'Knn': (0.45799915576192485, 0.6607795371498173, 0.5410122164048866, None), 'SVM': (0.5204727733220769, 0.6950394588500564, 0.5952208544532946, None), 'Random forest': (0.16842549598986914, 0.23170731707317074, 0.19506233194817893, None), 'XGBoost': (0.5479105107640354, 0.7008639308855291, 0.615020137408197, None)}


In [ ]:
print("Acurácia dos classificadores"+str(acuracia_values))

Acurácia dos classificadores
{'Regressao logistica': 0.8285392568328386, 'Knn': 0.8115467294503019, 'SVM': 0.8283345275872659, 'Random forest': 0.8265943289998976, 'XGBoost': 0.6596376292353363}


Analisando todos os classificadores, com o tratamento realizado nos dados, pode-se considerar que a classificação utilizando SVM e Regressão logística, apresentaram os melhores resultados, do ponto de vista de acurácia.